In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn_pandas import DataFrameMapper, gen_features, cross_val_score
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.decomposition import PCA, KernelPCA
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, SGDClassifier, Perceptron, Ridge
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib
from sklearn.metrics import *
from scipy import stats
import xgboost as xgb

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
np.random.seed(10)

In [10]:
df = pd.read_pickle('data/train1.pkl')

In [5]:
df.columns

Index(['click', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21'],
      dtype='object')

In [6]:
df.nunique()

click                     2
hour                     24
C1                        7
banner_pos                7
site_id                2865
site_domain            3394
site_category            22
app_id                 4154
app_domain              287
app_category             31
device_id            368962
device_ip           1078153
device_model           6098
device_type               4
device_conn_type          4
C14                     785
C15                       8
C16                       9
C17                     193
C18                       4
C19                      44
C20                     168
C21                      38
dtype: int64

In [12]:
df['hour_of_day'] = df['hour'].map(lambda x: int(str(x)[-2:]))
# df['day'] = df['hour'].map(lambda x: int(str(x)[4:6]))
# df['month'] = df['hour'].map(lambda x: int(str(x)[2:4]))


In [13]:
# pick colums with less possible categories to keep things simple-- since it will be a very large matrix
# after one hot encoding

df_cols = ['click', 'hour_of_day', 'C1', 'banner_pos',
       'site_category', 'app_category',
         'device_type', 'device_conn_type', 
       'C15', 'C16', 'C17', 'C18', 'C19', 'C21']

feature_cols = ['hour_of_day', 'C1', 'banner_pos',
       'site_category', 'app_category',
         'device_type', 'device_conn_type', 
       'C15', 'C16', 'C17', 'C18', 'C19', 'C21']

df = df[df_cols]


In [14]:
df.head()

,click,hour_of_day,C1,banner_pos,site_category,app_category,device_type,device_conn_type,C15,C16,C17,C18,C19,C21
0,0,0,1005,0,28905ebd,07d7df22,1,2,320,50,1722,0,35,79
1,0,0,1005,0,28905ebd,07d7df22,1,0,320,50,1722,0,35,79
2,0,0,1005,0,28905ebd,07d7df22,1,0,320,50,1722,0,35,79
3,0,0,1005,0,28905ebd,07d7df22,1,0,320,50,1722,0,35,79
4,0,0,1005,1,0569f928,07d7df22,1,0,320,50,2161,0,35,157


In [31]:
# mapper2 = DataFrameMapper([
#     (['C1'], LabelBinarizer()),
#     (['banner_pos'], LabelBinarizer()),
#     (['device_type'], LabelBinarizer()),
#     (['device_conn_type'], LabelBinarizer()),
#     (['C15'], LabelBinarizer()),
#     (['C16'], LabelBinarizer()),
#     (['C18'], LabelBinarizer()),
#     (['hour_of_day'], LabelBinarizer()),
#     (['day'], LabelBinarizer())
# ], input_df=True)


feature_def = gen_features(
    columns=feature_cols,
    classes=[LabelBinarizer])
# classes=[MinMaxScaler, LabelBinarizer])
    
mapper = DataFrameMapper(feature_def, input_df=True)
# feature_select = sklearn.feature_selection.SelectKBest(k=100)

In [43]:
X = df[feature_cols]
y = df.click

for col in X.columns:
    X[col].astype('str')

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [45]:
X_train.head()

,hour_of_day,C1,banner_pos,site_category,app_category,device_type,device_conn_type,C15,C16,C17,C18,C19,C21
2249607,10,1005,0,28905ebd,07d7df22,1,0,320,50,1882,3,35,13
234579,1,1005,0,3e814130,07d7df22,1,0,300,250,2295,2,35,23
1731229,8,1005,0,50e219e0,0f2161f8,1,0,320,50,2229,0,687,48
3625300,18,1005,0,f028772b,07d7df22,1,0,320,50,2374,3,39,23
2048166,9,1005,0,28905ebd,07d7df22,1,2,320,50,2502,0,35,221


In [46]:

pipe_forest = Pipeline([
    ('featurize', mapper),
    ('reduce_dim', KernelPCA(n_jobs=-1, kernel="rbf")),
    ('random_forest', RandomForestClassifier(n_jobs=-1, random_state=10, n_estimators=100))])

In [ ]:
pipe_forest.fit(X_train, y_train)
pipe_forest.predict(X_test, y_test)
report_rf = classification_report( y_test, y_prediction )
print(report_rf)

In [ ]:
init_test = np.round(cross_val_score(pipe, X=X_train , y=y_train, scoring='f1', n_jobs = -1), 2)

In [ ]:
from hyperopt import hp

# Parameter search space
space = {}

# One of (1,1), (1,2), or (1,3)
space['vec__ngram_range'] = hp.choice('vec__ngram_range', [(1,1), (1,2), (1,3)])

# Random integer in [1,3]
space['vec__min_df'] = 1+hp.randint('vec__min_df', 3)

# Uniform between 0.7 and 1
space['vec__max_df'] = hp.uniform('vec__max_df', 0.7, 1.0)
    
# One of True or False
space['vec__sublinear_tf'] = hp.choice('vec__sublinear_tf', [True, False])
    
# Random number between 50 and 100
space['kbest__percentile'] = hp.uniform('kbest__percentile', 50, 100)
    
# Random number between 0 and 1
space['clf__l1_ratio'] = hp.uniform('clf__l1_ratio', 0.0, 1.0)
      
# Log-uniform between 1e-9 and 1e-4
space['clf__alpha'] = hp.loguniform('clf__alpha', -9*np.log(10), -4*np.log(10))
      
# Random integer in 20:5:80
space['clf__n_iter'] = 20 + 5*hp.randint('clf__n_iter', 12)

In [4]:
pipeline = Pipeline([
    ('featurize', featurize(features)), 
    ('forest', RandomForestClassifier(n_estimators = 10))
])

pipeline.fit(X, y).predict(df_train)

param_grid = dict(forest__n_estimators = [2, 16, 32], forest__criterion = ['gini', 'entropy'])

grid_search = GridSearchCV(pipeline, param_grid=param_grid, scoring='accuracy')

best_pipeline = grid_search.fit(X, y).best_estimator_
best_pipeline.get_params()['forest']


NameError: name 'featurize' is not defined

In [29]:
X_train.shape

(36210029, 10)